In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [10]:
sns.set_style("whitegrid")

nltk.download('stopwords')

stop_words = set(stop_words.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'stop_words' is not defined

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/datasets/yasserh/amazon-product-reviews-dataset/7817_1.csv")

In [ ]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

In [ ]:
df['clean_review'] = df['reviews.text'].apply(clean_text)

In [ ]:
def label_sentiment(rating):
    if rating >= 4:
        return 1   
    elif rating == 3:
        return 0     
    else:
        return -1  

In [ ]:
df['sentiment'] = df['reviews.rating'].apply(label_sentiment)

In [ ]:
df['sentiment'].value_counts()

In [ ]:
sns.countplot(x='sentiment', data=df)
plt.title("Sentiment Distribution")
plt.show()

In [ ]:
plt.figure()
sns.countplot(x='reviews.rating', data=df)
plt.title("Rating Distribution")
plt.show()

In [ ]:
plt.figure()
sns.countplot(x='reviews.rating', data=df)
plt.title("Rating Distribution")
plt.show()

In [ ]:
df['review_length'] = df['clean_review'].apply(len)

plt.figure()
sns.histplot(df['review_length'], bins=50)
plt.title("Review Length Distribution")
plt.show()

In [ ]:
plt.figure()
sns.boxplot(x='sentiment', y='review_length', data=df)
plt.title("Review Length by Sentiment")
plt.show()

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

In [ ]:
X = vectorizer.fit_transform(df['clean_review'])
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
df[['reviews.text','reviews.rating']].isnull().sum()

In [ ]:
df = df.dropna(subset=['reviews.rating'])

In [ ]:
df[['reviews.text','reviews.rating']].isnull().sum()

In [ ]:
df['reviews.rating'] = df['reviews.rating'].astype(int)

In [ ]:
def label_sentiment(rating):
    if rating >= 4:
        return 1
    elif rating == 3:
        return 0
    else:
        return -1

In [ ]:
df['sentiment'] = df['reviews.rating'].apply(label_sentiment)

In [ ]:
df['sentiment'].value_counts()

In [ ]:
X = vectorizer.fit_transform(df['clean_review'])
y= df['sentiment']

In [ ]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2)
)

In [ ]:
X = vectorizer.fit_transform(df['clean_review'])
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
"Accuracy:", accuracy_score(y_test, y_pred)

In [ ]:
print("\nClassification Report:\n")
print("classification_report(y_test, y_pred)")

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
plt.figure()

In [ ]:
sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

In [ ]:
y_test_bin = label_binarize(y_test, classes=[-1, 0, 1])
y_score = model.predict_proba(X_test)

In [ ]:
plt.figure()

In [ ]:
for i, class_label in enumerate([-1, 0, 1]):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {class_label} (AUC = {roc_auc:.2f})")

In [ ]:
plt.plot([0,1], [0,1], linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

In [ ]:
from wordcloud import WordCloud

In [ ]:
positive_text = " ".join(df[df['sentiment'] == 1]['clean_review'])

In [ ]:
wordcloud = WordCloud(width=800, height=400).generate(positive_text)

In [ ]:
plt.figure()

In [ ]:
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.title("Positive Reviews WordCloud")
plt.show()

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
svm_model = LinearSVC(class_weight='balanced')
svm_model.fit(X_train, y_train)

In [ ]:
svm_pred = svm_model.predict(X_test)

In [ ]:
print("SVM Classification Report:\n")

In [ ]:
print(classification_report(y_test, svm_pred))

In [ ]:
print("Logistic Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))